In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 125

import seaborn as sns

import pandas as pd
import numpy as np
np.random.seed(0) # Giúp chạy các hàm của scikit-learn giống nhau mỗi lần chạy

import regex as re
import time # Dùng để sleep chương trình
from tqdm.notebook import tqdm # Hiện thanh progress cho đẹp :D
tqdm.pandas()

# Thư viện để request và parse HTML
import requests
from bs4 import BeautifulSoup

# Các thư viện liên quan tới ngôn ngữ và NLP
from pyvi import ViTokenizer # Thư viện NLP tiếng Việt
import gensim
import unicodedata # Thư viện unicode

# Trực quan hóa mô hình dự đoán văn bản
from lime import lime_text

# Dùng để lưu lại model
import pickle

# Thư viện liên quan của Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import feature_selection

# Tạo pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer, make_column_selector

# Các mô hình học
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier # Phương pháp bagging

In [8]:
# Thiết lập đường dẫn cho phần 1
dir_1 = "C:/Users/davin/Desktop/KHDL/"
# Thiết lập lại đường dẫn phần 1
dir_1_new = "C:/Users/davin/Desktop/KHDL/raw_data/"
# Thiết lập đường dẫn cho phần 2
dir_2 = "C:/Users/davin/Desktop/KHDL/pre_data/"
# Thiết lập đường dẫn cho phần 3
dir_3 = "C:/Users/davin/Desktop/KHDL/final_data/"

In [9]:
# Đọc từng file và cộng dồn vào series
class_stat = pd.Series(dtype='int64')
for i in tqdm(range(40)):
    df = pd.read_csv(dir_2 + f'crawling_{i}.csv')
    class_stat = class_stat.add(df['class'].value_counts(), fill_value=0)

# Sort lại và chuyển về int
class_stat = class_stat.sort_values(ascending=False).astype('int64')

# Lưu dữ liệu thành file để thuận tiện cho quá trình khám phá
class_stat.to_csv(dir_3 + "class_statistic.csv",encoding="utf-8", header=False)

  0%|          | 0/40 [00:00<?, ?it/s]

In [10]:
# Thống kê phân lớp toàn dữ liệu
class_stat = pd.read_csv(dir_3 + "class_statistic.csv", header=None)
class_stat.columns = ["class", "count"]
class_stat

,class,count
0,thời sự,23258
1,thế giới,17425
2,thể thao,14947
3,kinh doanh,13166
4,pháp luật,11357
...,...,...
106,cầu nối,1
107,chào sea games,1
108,chuyện pháp đình,1
109,box ảnh,1


In [11]:
# Phân lớp có số mẫu lớn hơn 20
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 2):
    display(class_stat[class_stat['count'] > 20])

,class,count
0,thời sự,23258
1,thế giới,17425
2,thể thao,14947
3,kinh doanh,13166
4,pháp luật,11357
5,sức khỏe,7863
6,giáo dục,7222
7,xe,6782
8,cần biết,6740
9,giải trí,6584


In [12]:
'''
Lớp bình luận, phản hồi, đường dây nóng, bút bi, bạn đọc làm báo là những nhóm có nội dung nhiều đặc thù (mang tính cá nhân cao) và khó phân loại nên buộc phải loại bỏ các nhóm này.
Lớp giả-thật, cần biết, media cũng là những lớp nội dung chứa nhiều lớp tin túc không thể phân loại chính xác cũng sẽ được loại bỏ.
Lớp khoa học chứa thông tin liên quan đến các ngành khoa học khác như sinh học, hóa học,... trong khi nhịp sống số và công nghệ chủ yếu chứ tin liên quan tới công nghệ thông tin. Nhóm quyết định tách lớp khoa học riêng với nhóm trên.
Lớp ẩm thực, xã hội tương đối phù hợp để tạo thành lớp mới nhưng thu thập được ít dữ liệu nên quyết định gộp chung với lớp đời sống.
'''

def class_extract(df):
    selected_class = ["thời sự", "phóng sự", "tin tức", "thế giới", "thể thao", " bóng đá", "word cup 2022", "sea games 31", "các môn khác", "kinh doanh", "việc làm", "mua sắm", "nhà đất", "đầu tư", "dự án", "thị trường", "pháp luật", "sức khỏe", "biết để khỏe", "giáo dục", "tuyển sinh", "nhịp sống học đường", "câu truyện giáo dục", "học hànhh", "định hướng và tư vấn nghề nghiệp", "cẩm nang việc làm", "tuổi trẻ start-up award", "xe", "giải trí", "đời sống", "nhịp sống trẻ", "du lịch", "cơ hội du lịch", "đi chơi", "doanh nghiệp", "thời tiết", "công nghệ", "chuyển đổi số, nhịp sống số", "khoa học", "văn hóa"]

    processed_df = df[df["class"].isin(selected_class)].copy()
    
    processed_df.loc[processed_df["class"].isin(["thời sự", "phóng sự", "tin tức"]), "class"] = "thời sự trong nước"
    processed_df.loc[processed_df["class"].isin(["du lịch", "cơ hội du lịch", "đi chơi"]), "class"] = "du lịch"
    processed_df.loc[processed_df["class"].isin(["thể thao", "bóng đá", "word cup 2022", "sea games 31", "các môn khác"]), "class"] = "thể thao"
    processed_df.loc[processed_df["class"].isin(["sức khỏe", "biết để khỏe"]), "class"] = "sức khỏe"
    processed_df.loc[processed_df["class"].isin(["kinh doanh", "tài chính", "mua sắm", "nhà đất", "đầu tư", "dự án", "thị trường", "doanh nghiệp"]), "class"] = "kinh doanh"
    processed_df.loc[processed_df["class"].isin(["giáo dục", "tuyển sinh", "nhịp sống học đường", "câu truyện giáo dục", "học hành"]), "class"] = "giáo dục"
    processed_df.loc[processed_df["class"].isin(["định hướng và tư vấn nghề nghiệp", "cẩm nang việc làm", "tuổi trẻ start-up award", "việc làm"]), "class"] = "việc làm"
    processed_df.loc[processed_df["class"].isin(["đời sống", "nhịp sống trẻ", "xã hội", "ẩm thực"]), "class"] = "đời sống"
    processed_df.loc[processed_df["class"].isin(["công nghệ", "chuyển đổi số", "nhịp sống số"]), "class"] = "công nghệ"
    
    processed_df.loc[processed_df["class"] == "giải trí", "class"] = "giải trí"
    processed_df.loc[processed_df["class"] == "xe", "class"] = "xe"
    processed_df.loc[processed_df["class"] == "thời tiết", "class"] = "thời tiết"
    processed_df.loc[processed_df["class"] == "khoa học", "class"] = "khoa học"
    processed_df.loc[processed_df["class"] == "văn hóa", "class"] = "văn hóa"
    processed_df.loc[processed_df["class"] == "pháp luật", "class"] = "pháp luật"
    processed_df.loc[processed_df["class"] == "thế giới", "class"] = "thời sự quốc tế"
    
    return processed_df

In [13]:
def covert_unicode(txt):
    if (type(txt) is not str):
        return txt
    return unicodedata.normalize('NFC', txt)
    
def remove_unnecessary(paragraph):
    temp=re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',paragraph)
    return re.sub(r'\s+', ' ', temp).strip()

def tokenize(sentence):
    return ViTokenizer.tokenize(sentence)

# Đọc dữ liệu stopword từ file
# (nguồn: https://github.com/stopwords/vietnamese-stopwords)
with open('C:/Users/davin/Desktop/KHDL/vietnamese-stopwords-dash.txt', encoding='utf-8') as f:
    lines = f.read().splitlines()
    
# Danh sách stopword
stopword = set(lines)

def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

def lower_str(string):
    return string.lower()

# --------------- Tạo pipeline ---------------

# Đóng gói các hàm thành các bước trong pipeline
unicode_std = FunctionTransformer(covert_unicode)
lowercase_conv = FunctionTransformer(lower_str)
vn_std = FunctionTransformer(covert_unicode)
rm_unescesary = FunctionTransformer(remove_unnecessary)
word_tokenize = FunctionTransformer(tokenize)
rm_stopwords = FunctionTransformer(remove_stopwords)

# pipeline xử lý dữ liệu cho cột content
preprocess_pipe_str = Pipeline([
    ("unicode_standardize", unicode_std),
    ("lowercase_conv", lowercase_conv),
    ("vn_standardize", vn_std),
    ("rm_unescesary", rm_unescesary),
    ("word_tokenize", word_tokenize),
    ("rm_stopwords", rm_stopwords)
])
# Tạo function transformer áp dụng tiền xử lý lên từng phần tử trong dãy
preprocess_list_str = FunctionTransformer(np.vectorize(preprocess_pipe_str.transform))

# pipeline xử lý dữ liệu cho cột class
class_preprocess_pipe_str = Pipeline([
    ("unicode_standardize", unicode_std),
    ("lowercase_conv", lowercase_conv),
    ("vn_standardize", vn_std)
])
# Tạo function transformer áp dụng tiền xử lý lên từng phần tử trong class
class_preprocess_list_str = FunctionTransformer(np.vectorize(class_preprocess_pipe_str.transform))

# Khai báo các hàm cho function transformer dành riêng cho từng lớp
def class_preprocess(data_df):
    data_df["class"] = class_preprocess_list_str.transform(data_df["class"])
    data_df = class_extract(data_df)
    return data_df
def content_preproc(data_df):
    data_df["content"] = preprocess_list_str.transform(data_df["content"])
    return data_df

# Khai báo các hàm cho function transformer
def drop_col(data_df):
    return data_df.drop(['links', 'title', 'description'],axis=1)

def drop_na(data_df):
    return data_df.dropna(axis=0, how="any")

def copy_df(data_df):
    return data_df.copy()

preprocess_pipe_df = Pipeline([
    ("drop_columns", FunctionTransformer(drop_col)),
    ("drop_na", FunctionTransformer(drop_na)),
    ("copy_df", FunctionTransformer(copy_df)),
    ("class_extract", FunctionTransformer(class_preprocess)),
    ("content_preproc", FunctionTransformer(content_preproc)),
    ("drop_na_fin", FunctionTransformer(drop_na))
])

In [14]:
# Hàm tiền xử lý dữ liệu
def batch_preprocess():
    total_files = 40
    
    for index in tqdm(range(total_files)):
        df=pd.read_csv(dir_2 + f'crawling_{index}.csv')
        processed_df = preprocess_pipe_df.fit_transform(df)
        processed_df.to_csv(dir_3 + f'crawling_{index}.csv', index=False)

batch_preprocess()

  0%|          | 0/40 [00:00<?, ?it/s]

In [6]:
# Đọc từng file và cộng dồn vào series
class_stat = pd.Series(dtype='int64')
for i in tqdm(range(40)):
    df = pd.read_csv(dir_3 + f'crawling_{i}.csv')
    class_stat = class_stat.add(df['class'].value_counts(), fill_value=0)

# Sort lại và chuyển về int
class_stat = class_stat.sort_values(ascending=False).astype('int64')

# Lưu dữ liệu thành file để thuận tiện cho quá trình khám phá
class_stat.to_csv(dir_3 + "final_class_statistic.csv",encoding="utf-8", header=False)

  0%|          | 0/40 [00:00<?, ?it/s]